# DONT RUN

- you dont need to execute this file yourself
- it would automatically get executed as part of the paginate [scripts](./../360-paginate_data/360-paginate_data.ipynb)
- more details [here](../readme.md)


# setup

In [ ]:
# magic commands

%run ../../100-set_parameters/100-set_parameters.ipynb

In [ ]:
# query parameters

with open('./../360-paginate_data/issue_params.json', 'r') as file:
    fetch_issue_parameters = json.load(file)

repository_name = fetch_issue_parameters["repository_name"]
owner_name = fetch_issue_parameters["owner_name"]
number_of_issues = fetch_issue_parameters["number_of_issues"]
end_cursor = fetch_issue_parameters["end_cursor"]

fetch_issue_parameters = {
    "repository_name": repository_name,
    "owner_name": owner_name,
    "number_of_issues": number_of_issues,
    "end_cursor_value": end_cursor
}

# fetch next 100 closed issues

In [ ]:
# store result

next_100_closed_issues = global_query(fetch_next_100_closed_issues, fetch_issue_parameters)

In [ ]:
next_100_closed_issues

## dictionary to dataframe

In [ ]:
# drill down
# take the dictionary result from previous step
# reach out to a particular level within that dictionary (edges in this case)
# store that inside a dataframe

df = pd.DataFrame(d['node'] for d in next_100_closed_issues['data']['repository']['issues']['edges'])
df

## comments

In [ ]:
# access comments

df['comments'] = df['comments'].str['edges']

In [ ]:
# explode comments

df_flat_comments = df.explode('comments')
df_flat_comments['comments'] = df_flat_comments['comments'].str['node'].str['body']

## labels

In [ ]:
# access labels
df_flat_comments['labels'] = df_flat_comments['labels'].str['edges']

In [ ]:
# explode labels
df_flat_comments_and_labels = df_flat_comments.explode('labels')
df_flat_comments_and_labels['labels'] = df_flat_comments_and_labels['labels'].str['node'].str['name']

## final dataframe

In [ ]:
# print to screen

df_flat_comments_and_labels

## insert data

In [ ]:
# insert into csv

df_flat_comments_and_labels.to_csv("340-next_100_closed_issues.csv")

In [ ]:
# insert into json

f = open("340-next_100_closed_issues.json", "w")
f.write(json.dumps(next_100_closed_issues))
f.close()